In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from budget.plot import plot_summary
from budget.summary import all_records, subs_future, summary
from budget.watcher import FileWatcher

pd.set_option('display.max_rows', 1000)

In [ ]:
# needs to be in a separate cell, see https://github.com/ipython/ipython/issues/11098
mpl.rcParams['figure.dpi'] = 100

In [ ]:
sheets = {}
fw = FileWatcher(os.environ['PYBUDGET_LEDGER'], sheets)
fw.run()

In [ ]:
fw.stop()

In [ ]:
TOTAL_START = sheets['balance'].iloc[8, 3]
records = all_records(sheets)

## Summary

In [ ]:
summ = summary(records, end=datetime.date(2020, 12, 31))
summ.round().astype(int)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
plot_summary(summ, ax)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9, 3))
(summ.sum(axis=1).loc[lambda x: x > 0].cumsum() + TOTAL_START).plot(ax=axes[0])
axes[0].grid(axis='y', which='major', ls='dotted')
axes[0].set_ylim(0, None)
summ.sum(axis=1).loc[lambda x: x > 0].plot(ax=axes[1])
summ.sum(axis=1).loc[lambda x: x > 0].rolling(6).mean().plot(
    ax=axes[1], c='C0', ls='dotted'
)
axes[1].grid(axis='y', which='major', ls='dotted')

## Postponed payments

In [ ]:
records.loc[
    lambda x: (x['charge_len'].fillna(0) > 1)
    & (x['charge_end'] > np.datetime64('2020-08-13'))
    & (x['per_month'] < -10)
].sort_values('per_month')

In [ ]:
summary(records.loc[lambda x: x['charge_len'].fillna(0) > 1]).round()

In [ ]:
fig, axes = plt.subplots(2, 1)
subs_future(records, end=datetime.date(2024, 1, 1)).plot(ax=axes[0])
subs_future(records, end=datetime.date(2024, 1, 1)).cumsum().plot(ax=axes[1])
final = subs_future(records, end=records['charge_end'].dropna().max()).sum()
axes[1].axhline(final, c='black', ls='dashed')
for ax in axes:
    ax.set_xticks(list(range(0, 6 * 12, 12)))
    ax.set_xticklabels(list(range(2019, 2025)))
    ax.grid(axis='x', which='major', ls='dotted')
    ax.grid(axis='y', which='major', ls='dotted')
    ax.set_ylim(0, None)